<a href="https://colab.research.google.com/github/xamevou/someJupyterNotebooks/blob/main/KeyAndAddressGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Keys and addresses in Bitcoin

Taken from example 5 of chaper 4 in Antonopoulos' book.

First, we need the bitcoin package.

In [30]:
# !pip install bitcoin   # Uncomment if needed
from __future__ import print_function
import bitcoin

## Generate a random private key

We generate a random private key, but it must be valid. In other words, it must be an integer between 1 and `bitcoin.N`. This number, in the standard secp256k1, is the size $p$ of the Finite Field $F_p$ where the elliptic curve $y^2=x^3+7$ is defined. Its value is: $2^{256} - 2^{32} - 2^9 - 2^8 - 2^7 - 2^6 - 2^4 - 1$.

*Note: In the book, example 4.3 is in cpp. If you want to reproduce that example here, change `bitcoin.random_key()` with the following string: `private_key = "038109007313a5807b2eccc082c8c3fbb988a973cacf1a7df9ce725c31b14776"`*

In [31]:
valid_private_key = False
while not valid_private_key:    
    private_key = bitcoin.random_key()
    decoded_private_key = bitcoin.decode_privkey(private_key, 'hex')
    valid_private_key = 0 < decoded_private_key < bitcoin.N

print("Private Key (hex) is: ", private_key)
print("Private Key (decimal) is: ", decoded_private_key)

Private Key (hex) is:  80558e5954b3cc2eb1b3a634f631d2849d9e88742c1e9cb145e9245b6f6a9de2
Private Key (decimal) is:  58047209075500954953828700918391785314477924381046197853910210055201587109346


### WIF
The private key is now converted to Wallet Import Format (WIF). Supposedly, the WIF format encoding allows easy copy of a big number.

In [32]:
wif_encoded_private_key = bitcoin.encode_privkey(decoded_private_key, 'wif')
print("Private Key (WIF) is: ", wif_encoded_private_key)

Private Key (WIF) is:  5JnohvtQqyTCQMiVgYqqQpNnZrdx1g75Ed8SSWd9g9yZZ27jqe3


We add suffix "01" to create a "compressed" private key.

In [33]:
compressed_private_key = private_key + '01'
print("Private Key Compressed (hex) is: ", compressed_private_key)

Private Key Compressed (hex) is:  80558e5954b3cc2eb1b3a634f631d2849d9e88742c1e9cb145e9245b6f6a9de201


Now we generate a WIF format from the compressed private key (WIF-compressed)

In [34]:
wif_compressed_private_key = bitcoin.encode_privkey(bitcoin.decode_privkey(compressed_private_key, 'hex'), 'wif')
print("Private Key (WIF-Compressed) is: ", wif_compressed_private_key)

Private Key (WIF-Compressed) is:  L1XB9eqvFAAdL6S9bZYYg87A8265oyboh8TqHbkZHpcegDzZ8D6n


# From Private Key to Public Key

We multiply the EC generator point G, `bitcoin.G`, with the private key, to get a public key point. The G point, if you are interested, is $(55066263022277343669578718895168534326250603453777594175500187360389116729240, 32670510020758816978083085130507043184471273380659243275938904335757337482424)$

In [35]:
public_key = bitcoin.fast_multiply(bitcoin.G, decoded_private_key)
print("Public Key (x,y) coordinates is:", public_key)

Public Key (x,y) coordinates is: (57119633812722101323447062300054051366774019088490393806290653102014648620090, 42028849852090837927267654500191728373960641645568041272725782315563543982261)


Now we encode that public key as hex. It is done by concatenation of prefix 04 plus x(in hex) plus y(in hex). 

In [36]:
print("x component in hex is: ", hex(public_key[0]))
print("y component in hex is: ", hex(public_key[1]))

hex_encoded_public_key = bitcoin.encode_pubkey(public_key, 'hex')
print("\nPublic Key (hex) is:", hex_encoded_public_key)

x component in hex is:  0x7e48912aa1b1dd3f15ef005c1a345d37d934c1d0b065b2621008b6553541143a
y component in hex is:  0x5ceb7c6bc6e52f084baf7700abf43b82945782abe1dbdd4b171a65d0948070b5

Public Key (hex) is: 047e48912aa1b1dd3f15ef005c1a345d37d934c1d0b065b2621008b6553541143a5ceb7c6bc6e52f084baf7700abf43b82945782abe1dbdd4b171a65d0948070b5


### On "compression"

The public key is a point, as you see, but we can manage with only the x component.
"Compression" makes reference to the process of taking only the x component of the (x,y) pair that forms the public key. Every use of "compression" refers to that process.

So now we compress the public key, adjusting prefix depending on whether $y$ is even or odd.

In [37]:
(public_key_x, public_key_y) = public_key
compressed_prefix = '02' if (public_key_y % 2) == 0 else '03'
hex_compressed_public_key = compressed_prefix + (bitcoin.encode(public_key_x, 16).zfill(64))
print("Compressed Public Key (hex) is:", hex_compressed_public_key)

Compressed Public Key (hex) is: 037e48912aa1b1dd3f15ef005c1a345d37d934c1d0b065b2621008b6553541143a


# Generation of the bitcoin address from the public key

At last, we generate the address that corresponds to the public key.

In [38]:
print("Generating address for the public key: ", public_key)
print("\nBitcoin Address (b58check) is:", bitcoin.pubkey_to_address(public_key))

Generating address for the public key:  (57119633812722101323447062300054051366774019088490393806290653102014648620090, 42028849852090837927267654500191728373960641645568041272725782315563543982261)

Bitcoin Address (b58check) is: 1PHyaL6EopaKAWimCedgy9R7YfRQajJC2R


The address is now compressed (meaning: it will be generated from the compressed public key, the one with x component only).

In [39]:
print("Generating address from the compressed public key: ", hex_compressed_public_key)

print("\nCompressed Bitcoin Address (b58check) is:", bitcoin.pubkey_to_address(hex_compressed_public_key))

Generating address from the compressed public key:  037e48912aa1b1dd3f15ef005c1a345d37d934c1d0b065b2621008b6553541143a

Compressed Bitcoin Address (b58check) is: 1AijAtwkVMvQvmFwKvKvYA619uAiF3bEwV


Please note that the address generated is different depending if we started with the (x,y) public key or with the compressed (x) one. The wallet software must take care for the compatibility between old non-compressed addresses and compressed ones.

And that's all !

## NOTE
As you see, this process proceeds in a one-to-one fashion: one seed, one private key, one public key, one address (compressed or not). Obviously we want more flexibility in our wallets. That's what the seed words are for: with the words, several seeds are generated, which lead to several private keys, etc. 

From the words to the keys, that should be a topic for another Notebook. 